In [1]:
from asyncore import read
import spacy
from spacy import displacy
from utils.utils import count_occurrences, get_entities, search_with_google, wiki_content
import json

nlp = spacy.load('it_core_news_sm')

with open(f'assets/test_sentences.txt', 'r', encoding='utf-8') as f:
    text = f.read() 

doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

In [2]:
sentence_list = list(doc.sents) 

# Opening JSON file
f = open('./assets/italian_cities.json')
italian_cities = json.load(f)
f.close()
# Count occurrences
counter = count_occurrences(doc, italian_cities, "name")  

# Max occurrence
context = max(counter, key=counter.get) 
print(context)

# Get entities without duplicates
searchable_entities = get_entities(doc, counter)

print(searchable_entities)
#print(searchable_entities)

# Search addresses with Google
search_with_google(searchable_entities, context)

Torino
['Novecento', 'Italia', 'Ottocento', 'Novecento.', 'Impresa Porcheddu', 'Esposizione generale italiana', 'Borgo Medievale', 'Vetreria Albano&Macario', 'Terrazza Solferino', 'Mobilificio Torinese F. Cesare Gandolfo', 'Albergo Rocciamelone di', 'Francia', 'Villino Raby', 'Casa Fenoglio-Lafleur', 'Casa Rossi-Galateri', 'Passalacqua', 'Casa Girardi', 'Cibrario 54.', 'Crocetta', 'Casa Maffei']
https://google.it/search?q=Novecento+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Italia+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Ottocento+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Novecento.+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Impresa+Porcheddu+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Esposizione+generale+italiana+Torino&hl=it
Address:  None
Activity:  None
https://google.it/search?q=Borgo+Medievale+Torino&hl=it
Address:  None
Activity:  None
ht

Status code 401 from http://dev.virtualearth.net/REST/v1/Locations: ERROR - 401 Client Error: Unauthorized for url: http://dev.virtualearth.net/REST/v1/Locations?q=453+Booth+Street%2C+Ottawa+ON&o=json&inclnb=1&key=409dd128da53470fa49b1af74e7ab227&maxResults=1


<[ERROR - 401 Client Error: Unauthorized for url: http://dev.virtualearth.net/REST/v1/Locations?q=453+Booth+Street%2C+Ottawa+ON&o=json&inclnb=1&key=409dd128da53470fa49b1af74e7ab227&maxResults=1] Bing - Geocode [empty]>
